In [2]:
import json, csv, string, os
import numpy as np
from scipy.stats import describe

trans_path = './bigtest/oyez/'
verbose = True

In [67]:
with open('./bigtest/set_dict.json') as f:
    set_dict = json.load(f)

In [68]:
def load_oyez(case, pth=trans_path):    
    f = open(pth+case+'.txt','r')
    k = f.readlines()
    f.close()
    oyez = []
    wd = []
    for u in k:
        if len(u)>=3: # check for filler lines
            t0, t1, spkr = u.split(' ')[0:3]
            text = u.split(spkr)[-1]
            oyez.append(text)
            temp = text.split(' ')
            stp = ((float(t1)-float(t0))/len(temp))
            seq = [float(t0)+(stp*i) for i in range(len(temp))]
            for idx, time in enumerate(seq):
                if temp[idx]!='':
                    wd.append([temp[idx], time, time+(stp-.0001)])       
    return oyez, wd

def clean_trans(lst):
    clean  = ''.join(lst)
    c = clean.translate(str.maketrans('', '', string.punctuation))
    c = c.replace('\n', '')
    c = c.lower()
    return c
   

In [69]:
# Segment Pulling Functions
def word_time_list(lst):
    start = []
    end = []
    for item in lst:
        start.append(item[1])
        end.append(item[2])
    return start, end

def find_nearest(array, value):
    return (np.abs(np.asarray(array) - value)).argmin()

def get_seg(st, e, name, start, end, trans=None):
    st_id = find_nearest(start, float(st))
    en_id = find_nearest(end, float(e)+float(st))
    if trans is not None:
        script = trans.split(' ')[st_id:en_id+1]
        return [name, ' '.join(script), (float(st), float(st)+float(e))]
    return st_id, en_id, name  

# Generate BERT Sequences

In [77]:
rttm_path = './rttm/' # gold standard diarization
#rttm_path = './rdsv/' # RDSV predicted diarization

In [78]:
to_BERT = {}
for wav in set_dict['t']:
    case = wav.split('.')[0]

    if 'rdsv' in rttm_path:
        with open(rttm_path+case+'_rdsv.rttm', newline='\n') as f:
            reader = csv.reader(f)
            case_diary = list(reader)        
    else:
        with open(rttm_path+case+'.rttm', newline='\n') as f:
            reader = csv.reader(f)
            case_diary = list(reader)

    oyez_toBERT = []
    for item in case_diary:
        temp = item[0].split(' ')
        if temp[7][-14:]=='scotus_justice' and float(temp[4])>3:
            oyez_toBERT.append(get_seg(temp[3], temp[4], temp[7], ostl, oetl, trans=''.join(oyez)))
            
    to_BERT[case]= {'OYEZ': oyez_toBERT}

In [79]:
if rttm_path.split('/')[1]=='rttm':
    with open("./toBERT_oyez.json", "w") as outfile: 
        json.dump(to_BERT, outfile)
else:
    with open("./toBERT_"+rttm_path.split('/')[1]+".json", "w") as outfile: 
        json.dump(to_BERT, outfile)